In [ ]:
# Object Detector Code

import os
import numpy as np
import matplotlib.pyplot as plt 
import glob
import shutil
import imutils
import cv2

import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.utils import np_utils

In [ ]:
#upload model
from google.colab import files

uploaded = files.upload()

In [ ]:
from keras.models import load_model
from keras.models import model_from_json

In [ ]:
# load json and create model
json_file = open('model_v2.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)

In [ ]:
# load weights into new model
model.load_weights("model_v2.h5")
print("Loaded model from disk")

# summarize model
model.summary()

In [ ]:
# import the necessary packages
from tensorflow.keras.applications.resnet import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.applications import imagenet_utils
from imutils.object_detection import non_max_suppression

In [ ]:
#detection helper functions
def sliding_window(image, step, ws):
    # slide a window across the image
    for y in range(0, image.shape[0] - ws[1], step):
        for x in range(0, image.shape[1] - ws[0], step):
            # yield the current window
            yield (x, y, image[y:y + ws[1], x:x + ws[0]])

In [ ]:
def image_pyramid(image, scale=1.5, minSize=(20, 20)):
    # yield the original image
    yield image

    # keep looping over the image pyramid
    while True:
        # compute the dimensions of the next image in the pyramid
        w = int(image.shape[1] / scale)
        image = imutils.resize(image, width=w)

        # if the resized image does not meet the supplied minimum
        # size, then stop constructing the pyramid
        if image.shape[0] < minSize[1] or image.shape[1] < minSize[0]:
            break

        # yield the next image in the pyramid
        yield image

In [ ]:
# initialize variables used for the object detection procedure
WIDTH,HEIGHT = 500,500
PYR_SCALE = 1.5
WIN_STEP = 30
ROI_SIZE = (100,100)
INPUT_SIZE = (100, 100)

In [ ]:
from google.colab import files

uploaded = files.upload()

#path = 'multiple_test_image.jpg'
path = 'apple_tree.jpg'
orig = cv2.cvtColor(cv2.imread(path),cv2.COLOR_BGR2RGB)   # convert to array
dim = (WIDTH, HEIGHT)
orig = cv2.resize(orig, dim)
(H, W) = orig.shape[:2]

In [ ]:
plt.imshow(orig)
plt.xlabel("Image size: ({},{})".format(orig.shape[0],orig.shape[1]))
plt.savefig("test_image.png")
files.download("test_image.png") 
plt.show()

In [ ]:
# initialize the image pyramid
pyramid = image_pyramid(orig, scale=PYR_SCALE, minSize=ROI_SIZE)
print(type(pyramid))

In [ ]:
# initialize two lists, one to hold the ROIs generated from the image
# pyramid and sliding window, and another list used to store the
# (x, y)-coordinates of where the ROI was in the original image
rois = []
locs = []

In [ ]:

from google.colab.patches import cv2_imshow

visualize_check = 1
count = 0

# loop over the image pyramid
for image in pyramid:
  
  # determine the scale factor between the *original* image
  # dimensions and the *current* layer of the pyramid
  scale = W / float(image.shape[1])

  # for each layer of the image pyramid, loop over the sliding
  # window locations
  for (x, y, roiOrig) in sliding_window(image, WIN_STEP, ROI_SIZE):
      # scale the (x, y)-coordinates of the ROI with respect to the
      # *original* image dimensions
      x = int(x * scale)
      y = int(y * scale)
      w = int(ROI_SIZE[0] * scale)
      h = int(ROI_SIZE[1] * scale)

      # take the ROI and pre-process it so we can later classify
      # the region using Keras/TensorFlow
      roi = cv2.resize(roiOrig, INPUT_SIZE)
      #roi = img_to_array(roi)
      #roi = preprocess_input(roi)
      roi = np.array(roi)
      roi = roi.astype('float32')/255

      # update our list of ROIs and associated coordinates
      rois.append(roi)
      locs.append((x, y, x + w, y + h))
      if count == 5:
        visualize_check = -1
      # check to see if we are visualizing each of the sliding
      # windows in the image pyramid
      if visualize_check > 0:
          # clone the original image and then draw a bounding box
          # surrounding the current region
          clone = orig.copy()
          cv2.rectangle(clone, (x, y), (x + w, y + h),
              (0, 255, 0), 2)

          # show the visualization and current ROI
          print("Visualization")            
          clone = cv2.cvtColor(clone,cv2.COLOR_RGB2BGR)   
          cv2_imshow(clone)
          print("ROI")
          roiOrig = cv2.cvtColor(roiOrig,cv2.COLOR_RGB2BGR)
          cv2_imshow(roiOrig)
          #cv2.waitKey(0)
          count = count + 1

In [ ]:
# convert the ROIs to a NumPy array
#rois = np.array(rois, dtype="float32")
rois = np.array(rois)

In [ ]:

# classify each of the proposal ROIs using ResNet and then show how
# long the classifications took
print("[INFO] classifying ROIs...")
preds = model.predict(rois)

In [ ]:
prediction_array = []
labels = ['Apple Granny Smith', 'Apple Red 1', 'Avocado', 'Banana Lady Finger', 'Cocos', 'Corn', 'Eggplant', 'Mandarine', 'Pepper Green', 'Strawberry']

for i in range(preds.shape[0]):
    pred_index = np.argmax(preds[i])
    prediction_label = labels[pred_index]
    probability = np.max(preds[i])
    prediction_array.append([prediction_label,probability])
    #display predictions 
    print([prediction_label,probability])

prediction_array = np.array(prediction_array)
print(prediction_array.shape)
#print(prediction_array[84])

In [ ]:
min_conf = 0.99

labels = {}

# loop over the predictions
for i in range(len(prediction_array)):
    # grab the prediction information for the current ROI
    (label, prob) = prediction_array[i]

    # filter out weak detections by ensuring the predicted probability
    # is greater than the minimum probabilitystac
    if eval(prob) >= min_conf:
        # grab the bounding box associated with the prediction and
        # convert the coordinates
        box = locs[i]

        # grab the list of predictions for the label and add the
        # bounding box and probability to the list
        L = labels.get(label, [])
        L.append((box, prob))
        labels[label] = L

print(labels.keys())

In [ ]:
from google.colab.patches import cv2_imshow

# loop over the labels for each of detected objects in the image
for label in labels.keys():

  # clone the original image so that we can draw on it
  print("[INFO] showing results for '{}'".format(label))
  clone = orig.copy()

  # loop over all bounding boxes for the current label
  for (box, prob) in labels[label]:
      # draw the bounding box on the image
      (startX, startY, endX, endY) = box
      cv2.rectangle(clone, (startX, startY), (endX, endY),
          (0, 255, 0), 2)

  # show the results *before* applying non-maxima suppression, then
  # clone the image again so we can display the results *after*
  # applying non-maxima suppression
  print('Before')
  clone = cv2.cvtColor(clone,cv2.COLOR_RGB2BGR)
  cv2_imshow(clone)
  clone = orig.copy()

  # extract the bounding boxes and associated prediction
  # probabilities, then apply non-maxima suppression
  boxes = np.array([p[0] for p in labels[label]])
  proba = np.array([p[1] for p in labels[label]])
  boxes = non_max_suppression(boxes, proba)

  # loop over all bounding boxes that were kept after applying
  # non-maxima suppression
  for (startX, startY, endX, endY) in boxes:
      # draw the bounding box and label on the image
      cv2.rectangle(clone, (startX, startY), (endX, endY),
          (0, 255, 0), 2)
      y = startY - 10 if startY - 10 > 10 else startY + 10
      cv2.putText(clone, label, (startX, y),
          cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 255, 0), 2)

  # show the output after apply non-maxima suppression
  print('After')
  clone = cv2.cvtColor(clone,cv2.COLOR_RGB2BGR)
  cv2_imshow(clone)
  #cv2.waitKey(0)